In [7]:
import numpy as np
import plotly.graph_objects as go


def gbm_prices(s0, mu, sigma, dt, n_steps):
    """
    Generate a price series using Geometric Brownian Motion (GBM).

    Parameters:
    s0 (float): Initial price
    mu (float): Drift coefficient (expected return)
    sigma (float): Volatility coefficient
    dt (float): Time step size
    n_steps (int): Number of time steps

    Returns:
    numpy.ndarray: Price series generated using GBM
    """
    # Initialize price series array
    prices = np.zeros(n_steps)
    prices[0] = s0

    # Generate price series using GBM
    for i in range(1, n_steps):
        z = np.random.standard_normal()
        prices[i] = prices[i - 1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z
        )

    return prices


# Example usage
s0 = 100.0  # Initial price
mu = 0.1  # Drift coefficient (expected return)
sigma = 0.2  # Volatility coefficient
dt = 1 / 252  # Time step size (assuming 252 trading days in a year)
n_steps = 1000  # Number of time steps

prices = gbm_prices(s0, mu, sigma, dt, n_steps)

# Plot price series
time_steps = np.arange(n_steps)
trace = go.Scatter(x=time_steps, y=prices, mode="lines", name="Price Series")
layout = go.Layout(
    title="Price Series Generated using Geometric Brownian Motion",
    xaxis=dict(title="Time Steps"),
    yaxis=dict(title="Price"),
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from typing import List, Tuple


def daily_returns(prices: np.ndarray) -> np.ndarray:
    daily_returns = np.diff(prices) / prices[:-1]
    return daily_returns


def prices_sigma(prices: np.ndarray) -> float:
    returns = daily_returns(prices)
    sigma = np.std(returns) * np.sqrt(252)  # Annualized volatility
    return sigma


def black_scholes_call(S: float, K: float, T: float, r: float, sigma: float) -> float:
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


def call_prices(
    prices: np.ndarray,
    strikes: List[float],
    days_to_expire: List[int],
    sigma: float,
    r: float = 0.05,
) -> pd.DataFrame:
    n_days = len(prices)

    results = []
    for strike in strikes:
        for days in days_to_expire:
            call_prices = []
            for i in range(n_days):
                S = prices[i]
                T = (days - i) / 252  # Update time to expiration
                if T >= 0:
                    call_price = black_scholes_call(S, strike, T, r, sigma)
                    call_prices.append(call_price)
                else:
                    call_prices.append(np.nan)  # Or any other appropriate value

            call_prices = np.array(call_prices)
            daily_pct_changes = np.diff(call_prices) / call_prices[:-1]
            daily_pct_changes = np.insert(
                daily_pct_changes, 0, np.nan
            )  # Insert 'NA' for day 0

            results.extend(
                zip(
                    np.repeat(days, n_days),
                    np.repeat(strike, n_days),
                    range(n_days),
                    call_prices,
                    daily_pct_changes,
                )
            )

    df = pd.DataFrame(
        results,
        columns=["Expire", "Strike", "Day", "Call Price", "Call Pct Change"],
    )

    return df


def gbm_prices(
    s0: float, mu: float, sigma: float, dt: float, n_steps: int
) -> np.ndarray:
    # Generate prices using Geometric Brownian Motion (GBM)
    z = np.random.normal(size=n_steps)
    prices = s0 * np.exp(
        np.cumsum((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    )
    return prices


# Generate prices using Geometric Brownian Motion (GBM)
s0 = 100.0  # Initial price
mu = 0.1  # Drift coefficient (expected return)
sigma = 0.2  # Volatility coefficient
dt = 1 / 252  # Time step size (assuming 252 trading days in a year)
n_steps = 1000  # Number of time steps
# prices = gbm_prices(s0, mu, sigma, dt, n_steps)


# Calculate call prices using Black Scholes model
r = 0.01  # Risk-free interest rate
sigma = 0.2  # Volatility
# sigma = prices_sigma(prices)
strikes = [80, 90, 100, 110, 120]  # Multiple strike prices
# days_to_expire = [30, 60, 90]  # Multiple expiration dates (in days)
days_to_expire = np.arange(0, 361, step=30)


df = call_prices(prices, strikes, days_to_expire, sigma, r)
df
# print(df.info())
df_pivot = df.pivot_table(
    index=["Expire", "Strike"], columns="Day", values="Call Pct Change", dropna=False
)

prices_returns = daily_returns(prices)
new_row_data = [np.nan] + list(prices_returns)
new_row_index = pd.MultiIndex.from_tuples(
    [("STOCK", "STOCK")], names=df_pivot.index.names
)
new_row_df = pd.DataFrame([new_row_data], index=new_row_index, columns=df_pivot.columns)

df_pivot = pd.concat([new_row_df, df_pivot])

# pd.options.display.float_format = "{:.2f}".format
pd.options.display.float_format = "{:.0%}".format
# df
# df_pivot

In [ ]:
from typing import Optional, Tuple, Literal


def get_strike_step(stock_price: float) -> float:
    if stock_price < 0:
        raise ValueError("Stock price cannot be negative")
    elif stock_price < 25:
        return 2.5
    elif stock_price < 200:
        return 5
    elif stock_price < 1000:
        return 10
    else:
        return 20


def select_option(
    cur_date: int,
    cur_stock_price: float,
    itm_atm_otm: Literal["out_of_the_money", "in_the_money", "at_the_money"],
    expiration_dates: list[int],
    min_days_before_expiration: int = 30,
) -> Optional[Tuple[int, float]]:
    if cur_stock_price < 0:
        raise ValueError("Stock price cannot be negative")

    strike_step = get_strike_step(cur_stock_price)
    print(f"strike_step: {strike_step}")

    # Find the suitable expiration date based on the current date and min_days_before_expiration
    min_expiration_date = cur_date + min_days_before_expiration
    expiration_date = next(
        (date for date in expiration_dates if date > min_expiration_date),
        None,
    )

    if expiration_date is None:
        return None

    # Find the OTM, ITM, or ATM strike price based on the option type
    if itm_atm_otm == "out_of_the_money":
        strike_price = (cur_stock_price // strike_step + 1) * strike_step
    elif itm_atm_otm == "in_the_money":
        strike_price = (cur_stock_price // strike_step) * strike_step
    elif itm_atm_otm == "at_the_money":
        strike_price = round(cur_stock_price / strike_step) * strike_step
    else:
        raise ValueError(f"Invalid option type: {itm_atm_otm}")

    return expiration_date, strike_price


current_date = 30
current_stock_price = 53.0
# option_moneyness = "out_of_the_money"
option_moneyness = "at_the_money"
expiration_dates = range(10, 360, 10)
min_days_before_expiration = 29

result = select_option(
    current_date,
    current_stock_price,
    option_moneyness,
    expiration_dates,
    min_days_before_expiration,
)

if result is None:
    print("No suitable option found.")
else:
    expiration_date, strike_price = result
    print(f"Selected option:")
    print(f"Expiration Date: Day {expiration_date}")
    print(f"Strike Price: {strike_price:.2f}")

In [ ]:
import numpy as np
from scipy.stats import norm
from typing import Literal, Optional, Tuple, Callable


def default_rollover_strategy(
    cur_date: int,
    cur_stock_price: float,
    option_position: Optional[Tuple[int, float]],
    expiration_dates: list[int],
    itm_atm_otm: Literal["out_of_the_money", "in_the_money", "at_the_money"],
    min_days_before_expiration: int = 30,
) -> Optional[Tuple[int, float]]:
    if option_position is None:
        return select_option(
            cur_date,
            cur_stock_price,
            itm_atm_otm,
            expiration_dates,
            min_days_before_expiration,
        )
    else:
        expiration_date, _ = option_position
        days_before_expiration = expiration_date - cur_date
        if days_before_expiration <= min_days_before_expiration:
            return select_option(
                cur_date,
                cur_stock_price,
                itm_atm_otm,
                expiration_dates,
                min_days_before_expiration,
            )
        else:
            return option_position


def backtest_call_strategy(
    stock_prices: list[float],
    dates: list[int],
    expiration_dates: list[int],
    r: float,
    sigma: float,
    rollover_strategy: Callable[
        [
            int,
            float,
            Optional[Tuple[int, float]],
            list[int],
            Literal["out_of_the_money", "in_the_money", "at_the_money"],
            int,
        ],
        Optional[Tuple[int, float]],
    ] = default_rollover_strategy,
) -> float:
    portfolio_value = 0
    option_position = None

    for i in range(len(dates)):
        cur_date = dates[i]
        cur_stock_price = stock_prices[i]

        if option_position is None or cur_date >= option_position[0]:
            if option_position is not None:
                expiration_date, strike_price = option_position
                payoff = max(cur_stock_price - strike_price, 0)
                portfolio_value += payoff
            option_position = rollover_strategy(
                cur_date,
                cur_stock_price,
                option_position,
                expiration_dates,
                "out_of_the_money",
            )
            if option_position is not None:
                expiration_date, strike_price = option_position
                time_to_expiration = (expiration_date - cur_date) / 365
                option_price = black_scholes_call(
                    cur_stock_price, strike_price, time_to_expiration, r, sigma
                )
                portfolio_value -= option_price

    return portfolio_value